# Retrieve total aerosol optical depth from the Atmosphere Data Store and visualise 2019 anomalies in Australia

<img src="https://ads.atmosphere.copernicus.eu/themes/custom/baby_blue/logo.svg" width=140> 
This notebook demonstrates how to retrieve NetCDF data from the Atmosphere Data Store (https://ads.atmosphere.copernicus.eu/) (ADS) and anomaly map using xarray and Cartopy libraries.

First, let's define some general purpose variables

In [ ]:
DATADIR = '../data'
DOWNLOAD = False
years = [str(y) for y in range(2003, 2020)]

Next, we will retrieve the data from the ADS by using the cdsapi Python library. 

https://ads.atmosphere.copernicus.eu/api-how-to 

**To be able to run the data retrieval example below you will need to register on the Atmosphere Data Store (https://ads.atmosphere.copernicus.eu/) and then visit this page: https://ads.atmosphere.copernicus.eu/api-how-to to obtain the API authentication key.**

If you want to speed up the process, you can use the files which are already available on the LTPY training platform.

In [ ]:
if DOWNLOAD:

    import cdsapi
    # to suppress SSL warnings
    import urllib3
    urllib3.disable_warnings()

    # enter your ADS authentication key
    URL = "https://ads.atmosphere.copernicus.eu/api/v2"
    KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    c = cdsapi.Client(url=URL, key=KEY)

    c.retrieve('cams-global-reanalysis-eac4-monthly',
    {
       'format': 'netcdf',
       'variable': 'total_aerosol_optical_depth_550nm',
       'year': years,
       'month': '12',
       'product_type': 'monthly_mean',
    },
    f'{DATADIR}/cams_aod_December.nc')


Let's inspect one file

In [ ]:
import xarray as xr
ds = xr.open_dataset(f'{DATADIR}/cams_aod_December.nc')
print(ds)

Let's compute AOD550 December climatology based on 2003-2018 CAMS Reanalysis

In [ ]:
clim = ds.sel(time=ds.time.dt.year < 2019).mean("time")
print(clim)
this_year = ds.sel(time=ds.time.dt.year == 2019).mean("time")
print(this_year)

In [ ]:
import matplotlib.pyplot as plt

import cartopy
import cartopy.crs as ccrs

map_proj = ccrs.PlateCarree()
BORDER_COLOUR = "#cccccc"

f, ax = plt.subplots(figsize=(16, 7), subplot_kw=dict(projection=map_proj))

anomaly = this_year - clim
p = anomaly.aod550.plot(ax=ax, transform=ccrs.PlateCarree(),cmap="RdBu_r", vmin=-0.5, vmax=0.5)
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS, color=BORDER_COLOUR)
ax.coastlines(color=BORDER_COLOUR)
ax.set_extent((105, 160, -50, 0))
ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree() )
plt.title("CAMS reanalysis total aerosol optical depth at 550nm\nDecember 2019 anomaly")
plt.show()